In [2]:
%pip install lightgbm

  Using cached lightgbm-4.3.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lightgbm: filename=lightgbm-4.3.0-py3-none-macosx_14_0_arm64.whl size=1408370 sha256=a9968b9a2c367d136606d866aa642f53b4547434505946121e219619e134d57f
  Stored in directory: /Users/TrangNgan/Library/Caches/pip/wheels/5f/b6/8e/2147c3932dcf4c69975fca6144cec78ef125bedd717ba975f3
Successfully built lightgbm
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
def loadfile(filename):
    with open(f'{filename}.pickle', 'rb') as fp:
        data = pickle.load(fp)
    return data

In [4]:
import numpy as np

X_train = loadfile('X_train')
X_test = loadfile('X_test')
X_val = loadfile('X_val')
y_train = loadfile('y_train')
y_test = loadfile('y_test')
y_val = loadfile('y_val')

In [5]:
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import fbeta_score

model = LGBMClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_val)
print(classification_report(y_val, predict))
balanced_accuracy_score(y_val, predict)
fbeta_score(y_val, predict, average='weighted', beta=2)

[LightGBM] [Info] Number of positive: 53148, number of negative: 4715
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006421 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1808
[LightGBM] [Info] Number of data points in the train set: 57863, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.918514 -> initscore=2.422332
[LightGBM] [Info] Start training from score 2.422332
              precision    recall  f1-score   support

           0       0.76      0.17      0.28      1600
           1       0.93      1.00      0.96     17688

    accuracy                           0.93     19288
   macro avg       0.85      0.58      0.62     19288
weighted avg       0.92      0.93      0.91     19288



0.9169879790568953

In [6]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# Assuming X_train and y_train are your training data and labels
# X_train.shape should be (n_samples, n_features)
# y_train.shape should be (n_samples,)

# Define the undersampling ratio
desired_ratio = {0: np.sum(y_train == 0), 1: int(4*np.sum(y_train == 0))}

# Random undersampling
undersampler = RandomUnderSampler(sampling_strategy=desired_ratio, random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

In [7]:
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import fbeta_score

model = LGBMClassifier()
model.fit(X_train_resampled, y_train_resampled)
predict = model.predict(X_val)
print(classification_report(y_val, predict))
balanced_accuracy_score(y_val, predict)
fbeta_score(y_val, predict, average='weighted', beta=2)

[LightGBM] [Info] Number of positive: 18860, number of negative: 4715
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002238 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1744
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.800000 -> initscore=1.386294
[LightGBM] [Info] Start training from score 1.386294
              precision    recall  f1-score   support

           0       0.46      0.37      0.41      1600
           1       0.94      0.96      0.95     17688

    accuracy                           0.91     19288
   macro avg       0.70      0.67      0.68     19288
weighted avg       0.90      0.91      0.91     19288



0.9104832043105273

In [ ]:
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [200, 300, 400],
    'max_depth': [5, 6, 7],
    'min_child_weight': [50, 100, 200, 300, 400],
    'reg_alpha': [0.1, 0.3, 0.5, 0.7, 1.0],
    'reg_lambda': [0.1, 0,3, 0.5, 0.7, 1.0]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring= 'f1_micro')
grid_search.fit(X_train_resampled, y_train_resampled)
print(grid_search.best_params_)

In [9]:
model = LGBMClassifier(learning_rate=0.1, max_depth=6, min_child_weight=50, n_estimators=300, reg_alpha=1.0, reg_lambda=1.0) #param o tren kia
model.fit(X_train_resampled, y_train_resampled)
predict = model.predict(X_val)
print(classification_report(y_val, predict))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 18860, number of negative: 4715
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000930 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1744
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.800000 -> initscore=1.386294
[LightGBM] [Info] Start training from score 1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 